In [1]:
import os
import pandas as pd

In [2]:
tables_to_join_with_path = "./csv/tables_to_join/"
# tables_to_join_with_path = "./csv/to_test/" # for testing
schema_augmented_path = "./csv/schema_augmentated.csv"
# schema_augmented_path = "./csv/test_schema_augmentated.csv" # for testing

schema_linked_to_join_path = "../record_linkage/csv/schema_linked.csv"

c'è la tabella 21a che ha righe duplicate quindi non la usiamo

In [3]:
col_idTable_to_join_with = ["Security_5a7098f0c868a712fd740a7b", "Company_5a708e29c868a712fd72e48d", "Airline_5a70aa27c868a712fd75b3a4", "Company_5a709ca3c868a712fd746a77", "Name_5a708f2dc868a712fd72f7f0", "Name_5a7098e5c868a712fd740889", "Make_5a708ca3c868a712fd72c735", "Company_5a70ab31c868a712fd75cc0e", "Name_5a70928ec868a712fd7343db", "Airline_5a708f2fc868a712fd72f81c", "Company_5a70a301c868a712fd7511ce", "English_5a708b9fc868a712fd72ba3b", "Name_5a708d9fc868a712fd72d9e4", "Name_5a70a031c868a712fd74c51c", "Company_5a708b7ac868a712fd72b7c6", "Company_5a7098d3c868a712fd7406d5", "Manufacturer_5a70d572c868a712fd79767a", "English_5a708b9fc868a712fd72ba3c", "Company_5a70a2bac868a712fd75071a", "Name_5a708e81c868a712fd72ec57", "Company_5a709467c868a712fd737622", "Company_5a70943dc868a712fd73721a", "English_5a708b9fc868a712fd72ba36", "Manufacturer_5a709ec0c868a712fd749c60"]

In [4]:
schema_linked_to_join = pd.read_csv(schema_linked_to_join_path)
schema_linked_to_join = schema_linked_to_join.drop("id", axis=1)

/var/folders/c2/dnpzswqd7ll8qj2knwvlk3v80000gn/T/ipykernel_4285/1084007032.py:1: DtypeWarning: Columns (5,10,11,12,13) have mixed types. Specify dtype option on import or set low_memory=False.
  schema_linked_to_join = pd.read_csv(schema_linked_to_join_path)


In [5]:
from valentine import valentine_match, valentine_metrics
from valentine.algorithms import Coma
from valentine.algorithms import Cupid
import os
import multiprocessing as mp
from multiprocessing import Pool
import time
import pandas as pd
from compute_match import *

In [6]:
# Funzione con cui si calcola il matching parallelizzando su più processori
def calculate_match_multithread(tuples):
    result = [] # result è una lista di tuple (ove una tupla è il risultato della funzione mappata)
    # Creo un pool per l'esecuzione multi processore (per velocizzare il matching tra tutte le coppie di dataframes)
    with Pool() as pool:
        # Ogni pool esegue la funzione (primo parametro), passandogli le tuple (secondo parametro)
        result = pool.map(calculate_match_coma_schema, tuples)
    return result

# funzione che riporta un dataframe contenente le informazioni circa tutti i match passati come parametro
# @param result, corrisponde al risultato del matching (i.e. nome dataset1, nome dataset2, il valore del matching di valentine)
def matches_to_table(result):
    matches = pd.DataFrame()
    # Per ogni risultato ottenuto dal matching parallelizzato
    for match in result:
    # Prendo il valore di matching ritornato dalla funzione valentine_match 
    # e.g. match[2] = ((table_1, 'Cited by'),(table_2, 'Cited by')): 0.83
        for key in match[2].keys():
            # e.g. key = (table_1, 'Cited by'),(table_2, 'Cited by')
            d = dict()
            d["table_1"] = match[0]
            d["table_2"] = match[1]
            d["column_table_1"] = key[0][1]
            d["column_table_2"] = key[1][1]
            d["match_value"] = match[2][key]
            # Aggiungiamo al dataframe l'ennupla
            matches = pd.concat([matches, pd.DataFrame(d, index=[0])], ignore_index=True)
    return matches

In [7]:
data_frames = {}
for file in os.listdir(tables_to_join_with_path):
    table_id = file.split("_")[-1].split(".")[0]
    data_frames[table_id] = pd.read_csv(tables_to_join_with_path + file)


In [8]:
# Creazione di tutte le coppie (schema-mediato, dataset)
matches_with_mediated = {}
all_mediatedNDbs_couples = []
for dbId in data_frames:
    dfl = schema_linked_to_join
    dfr = data_frames[dbId]
    # le tuple da passare al metodo di matching = nome dataset1, nome dataset2, dataframe1, dataframe2
    all_mediatedNDbs_couples.append(('mediated_schema', dbId, dfl, dfr))

In [9]:
# Matches con lo schema mediato
result = calculate_match_multithread(all_mediatedNDbs_couples)

In [10]:
matches = matches_to_table(result)

In [11]:
pd.DataFrame(matches).to_csv("./csv/matches_with_mediated_N_wikitables.csv")

In [12]:
colsTableMatches = pd.read_csv("./csv/matches_with_mediated_N_wikitables.csv")
tables_to_columns = dict()
# Per tutte le tabelle in match nella colonna table 2 vado a prendermi
# le colonne con match >= 0.5 con una colonna dello schema mediato
for table_name in set(colsTableMatches['table_2']):
    # Prende le colonne con table 2 uguale a table name
    table = colsTableMatches.loc[colsTableMatches['table_2'] == table_name]
    columns = set()
    for ind in table.index:
        if table['match_value'][ind] >= 0.4:
            columns.add((table['column_table_1'][ind], table['column_table_2'][ind]))
    tables_to_columns[table_name] = columns

In [13]:
nan = ["nan", "Nan", "NaN", "Not found", "Not Found", "not found", "not Found", "NULL", "null", "Null", "None", "none"]

In [14]:
for file in os.listdir(tables_to_join_with_path):
    # leggo il db corrente con cui fare un eventuale join
    curr_db_to_join_with = pd.read_csv(tables_to_join_with_path + file)

    # metto le colonne della tabella da joinare in lower case
    curr_db_to_join_with.columns = map(str.lower, curr_db_to_join_with.columns)
    
    # id table della tabella appena letta
    table_id = file.split("_")[-1].split(".")[0]

    ## prendo i match della tabella corrente con lo schema mediato
    table_to_columns_matches = tables_to_columns[table_id]

    joined = False
    # controllo che questa tabella sia tra le tabelle da joinare
    for col_dbId in col_idTable_to_join_with:
        col_N_idTable = col_dbId.split("_") # colonna e id
        curr_id_table = col_N_idTable[1] # id

        # è una delle tabelle con cui devo joinare
        if table_id == curr_id_table:

            column_to_join = col_N_idTable[0].lower() # colonna cu cui fare il join

            to_join_after = curr_db_to_join_with # il join lo faccio prima con un dataset a cui tolgo le colonne già presenti nello schema
            column_to_join_after = [] # colonne droppate perché già presenti nello schema, i cui dati verranno aggiunti dopo
            
            # per ogni colonna nel dataset da joinare
            for colDB in curr_db_to_join_with.columns:
                # se la colonna non è quella su cui fare il join ed è già presente nello schema
                if((colDB != column_to_join) and (colDB in schema_linked_to_join.columns)):

                    # droppo la colonna che poi aggiungerò in seguito
                    column_to_join_after.append(colDB)
                    to_join_after = to_join_after.drop(colDB, axis=1)
                
                ## altrimenti se la colonna non è nello schema e non è quella di join
                elif((colDB != column_to_join) and (colDB not in schema_linked_to_join.columns)):
                    ## se la colonna è in match con una colonna dello schema, allora la aggiungo dopo
                    for (colSchema, colTable) in table_to_columns_matches:
                        if(colDB==colTable.lower()):
                            ## aggiunta nelle colonne da joinare dopo
                            column_to_join_after.append(colDB)
                            ## e poi droppata
                            to_join_after = to_join_after.drop(colDB, axis=1)

            # modifico il nome della colonna su cui fare join => "name"
            to_join_after = to_join_after.rename(columns={column_to_join: "name"})

            # eseguo il merge sul campo "name"
            schema_linked_to_join = schema_linked_to_join.merge(to_join_after, on="name", how="outer", suffixes=["",""])

            # se ci sono colonne droppate allora le inserisco
            if(len(column_to_join_after) != 0):
                # riempio le colonne droppate di tutte le righe corrispondenti a quelle presenti nel dataset da joinare
                for nameCompany in curr_db_to_join_with[column_to_join]:
                    # id righe
                    idRowToJoin = curr_db_to_join_with.loc[curr_db_to_join_with[column_to_join]==nameCompany].index[0] # id della riga dell'azienda da joinare nel dataset da joinare
                    idRow = schema_linked_to_join.loc[schema_linked_to_join["name"]==nameCompany].index[0] # id della riga della stessa azienda ma nello schema

                    # riempio tutte le colonne droppate
                    for col in column_to_join_after:
                        ## se la colonna in questione è in match con una colonna dello schema
                        ## metto il contenuto dentro la colonna con cui sta in match
                        for (colSchema, colTable) in table_to_columns_matches:
                            if(col==colTable.lower()):
                                curr_data = schema_linked_to_join.iloc[idRow][colSchema] # dato presente nella cella corrente dello schema
                                # se il dato è vuoto allora lo aggiorno altrimenti mantengo quello già presente (non faccio nulla)
                                if(pd.isnull(curr_data) or (curr_data in nan)):
                                    schema_linked_to_join.loc[schema_linked_to_join["name"]==nameCompany, colSchema] = curr_db_to_join_with.iloc[idRowToJoin][col]
                                    already_joined = True

                        ## altrimenti aggiungo la colonna normalmente
                        # per tutte le colonne prima droppate e ora da aggiungere
                        # setto il dato nella cella dentro lo schema linked
                        curr_data = schema_linked_to_join.iloc[idRow][colSchema] # dato nella cella corrente
                        # se il dato non è stato inserito e la cella non è vuota allora aggiungo il dato, altrimenti no
                        if(not already_joined and (pd.isnull(curr_data) or (curr_data in nan))):
                            schema_linked_to_join.loc[schema_linked_to_join["name"]==nameCompany, col] = curr_db_to_join_with.iloc[idRowToJoin][col]
            
            joined = True # join completato

    if(not joined):
        print(file, "not joined")


# con stampe per testing

In [15]:
# for file in os.listdir(tables_to_join_with_path):
#     # leggo il db corrente con cui fare un eventuale join
#     curr_db_to_join_with = pd.read_csv(tables_to_join_with_path + file)

#     # metto le colonne della tabella da joinare in lower case
#     curr_db_to_join_with.columns = map(str.lower, curr_db_to_join_with.columns)
    
#     # id table della tabella appena letta
#     table_id = file.split("_")[-1].split(".")[0]

#     ## prendo i match della tabella corrente con lo schema mediato
#     table_to_columns_matches = tables_to_columns[table_id]
#     print("Matches:", table_to_columns_matches)

#     joined = False
#     # controllo che questa tabella sia tra le tabelle da joinare
#     for col_dbId in col_idTable_to_join_with:
#         col_N_idTable = col_dbId.split("_") # colonna e id
#         curr_id_table = col_N_idTable[1] # id

#         # è una delle tabelle con cui devo joinare
#         if table_id == curr_id_table:
#             print("Current db:", file)
#             display("To join with (lower):", curr_db_to_join_with)

#             column_to_join = col_N_idTable[0].lower() # colonna cu cui fare il join
#             print("Column to join:", column_to_join)

#             to_join_after = curr_db_to_join_with # il join lo faccio prima con un dataset a cui tolgo le colonne già presenti nello schema
#             column_to_join_after = [] # colonne droppate perché già presenti nello schema, i cui dati verranno aggiunti dopo
            
#             # per ogni colonna nel dataset da joinare
#             for colDB in curr_db_to_join_with.columns:
#                 # se la colonna non è quella su cui fare il join ed è già presente nello schema
#                 if((colDB != column_to_join) and (colDB in schema_linked_to_join.columns)):
#                     print("Da aggiungere dopo (già presente):", colDB)
#                     # droppo la colonna che poi aggiungerò in seguito
#                     column_to_join_after.append(colDB)
#                     to_join_after = to_join_after.drop(colDB, axis=1)
                
#                 ## altrimenti se la colonna non è nello schema e non è quella di join
#                 elif((colDB != column_to_join) and (colDB not in schema_linked_to_join.columns)):
#                     ## se la colonna è in match con una colonna dello schema, allora la aggiungo dopo
#                     for (colSchema, colTable) in table_to_columns_matches:
#                         if(colDB==colTable.lower()):
#                             print("Da aggiugnere dopo:", colDB, "matcha con", colSchema)
#                             ## aggiunta nelle colonne da joinare dopo
#                             column_to_join_after.append(colDB)
#                             ## e poi droppata
#                             to_join_after = to_join_after.drop(colDB, axis=1)

#             # modifico il nome della colonna su cui fare join => "name"
#             to_join_after = to_join_after.rename(columns={column_to_join: "name"})
#             display("After renaming in \"name\"", to_join_after)

#             # eseguo il merge sul campo "name"
#             schema_linked_to_join = schema_linked_to_join.merge(to_join_after, on="name", how="outer", suffixes=["",""])
#             display("Prima del joining con drop", schema_linked_to_join)

#             # se ci sono colonne droppate allora le inserisco
#             if(len(column_to_join_after) != 0):
#                 print("Devo aggiungere le colonne droppate")
#                 # riempio le colonne droppate di tutte le righe corrispondenti a quelle presenti nel dataset da joinare
#                 for nameCompany in curr_db_to_join_with[column_to_join]:
#                     print("Devo aggiungere all'azienda", nameCompany)
#                     # id righe
#                     idRowToJoin = curr_db_to_join_with.loc[curr_db_to_join_with[column_to_join]==nameCompany].index[0] # id della riga dell'azienda da joinare nel dataset da joinare
#                     idRow = schema_linked_to_join.loc[schema_linked_to_join["name"]==nameCompany].index[0] # id della riga della stessa azienda ma nello schema

#                     # riempio tutte le colonne droppate
#                     for col in column_to_join_after:
#                         ## se la colonna in questione è in match con una colonna dello schema
#                         ## metto il contenuto dentro la colonna con cui sta in match
#                         for (colSchema, colTable) in table_to_columns_matches:
#                             if(col==colTable.lower()):
#                                 curr_data = schema_linked_to_join.iloc[idRow][colSchema]
#                                 print("Aggiungo colonne in match", colTable, "con", colSchema)
#                                 # se il dato è vuoto allora lo aggiorno altrimenti mantengo quello già presente
#                                 if(pd.isnull(curr_data) or (curr_data in nan)):
#                                     schema_linked_to_join.loc[schema_linked_to_join["name"]==nameCompany, colSchema] = curr_db_to_join_with.iloc[idRowToJoin][col]
#                                     already_joined = True
#                                     print("Dopo aggiunta del matching alla colonna", colSchema, "del valore", curr_db_to_join_with.iloc[idRowToJoin][col])
#                                     display(schema_linked_to_join.iloc[idRow])
#                                 else:
#                                     print("Il dato già c'è nella colonna", colSchema, ":", curr_data)

#                         ## altrimenti aggiungo la colonna normalmente
#                         # per tutte le colonne prima droppate e ora da aggiungere
#                         # setto il dato nella cella dentro lo schema linked
#                         curr_data = schema_linked_to_join.iloc[idRow][colSchema] # dato nella cella corrente
#                         # se il dato non è stato inserito e la cella non è vuota allora aggiungo il dato
#                         if(not already_joined and (pd.isnull(curr_data) or (curr_data in nan))):
#                             schema_linked_to_join.loc[schema_linked_to_join["name"]==nameCompany, col] = curr_db_to_join_with.iloc[idRowToJoin][col]
#                             display("Dopo aggiunta colonna già presente", schema_linked_to_join.iloc[idRow])
#                         elif(not already_joined): 
#                             print("Il dato già c'è nella colonna", colSchema, ":", curr_data)
            
#             joined = True # join completato

#     if(not joined):
#         print(file, "not joined")


In [16]:
schema_linked_to_join.to_csv(schema_augmented_path)

In [17]:
schema_linked_to_join

,name,country,market cap,founded year,employees,industry,sector,ceo,revenue,stock,...,russian,notes,profits (billion $),assets (billion $),market value (billion $),countries listed,products / solutions,tse,rōmaji,japanese
0,superit srl,NaN,NaN,NaN,NaN,finance insurance and real estate,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,send to me srl,NaN,NaN,NaN,NaN,services,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,intesa sanpaolo vita spa,NaN,NaN,NaN,NaN,finance insurance and real estate,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,totalenergies petrochemicals & refining sa nv,NaN,NaN,NaN,NaN,wholesale trade,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,sanastera spa,NaN,NaN,NaN,NaN,finance insurance and real estate,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
74187,Yageo,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
74188,Yang Ming Marine Transport Corporation,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
74189,Yeh-Chiang Technology,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
74190,Yulon,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
